In [1]:
import requests
from bs4 import BeautifulSoup
import sqlite3
import re
import time

In [2]:
class Recipes:
    def __init__(self, link, photo_dish, name_dish, count_portion, cook_time, ingredients, calories):
        self.link = link 
        self.photo_dish = photo_dish
        self.name_dish = name_dish 
        self.count_portion = count_portion  
        self.cook_time = cook_time
        self.ingredients = ingredients 
        self.calories = calories

In [3]:
class Crawler:
    def __init__(self, url):
        self.url = url
        
        
    def get_soup(self):    
        return BeautifulSoup(requests.get(self.url).text, "html.parser")

In [4]:
class Parser:
    def __init__(self, html):
        self.html = html
        
    
#получение информации с одной страницы
#выгрузка данных в БД

    def get_ingredients(self):
        all_ingredients = {}
        name_ingredients = []
        
        for ing in self.html.findAll('div', class_='emotion-bjn8wh'):  
            name_ingredients.append(ing.text)

        for i in range (len(name_ingredients)):
            all_ingredients[name_ingredients[i]] = self.html.findAll('span', class_='emotion-bsdd3p')[i].text

        return all_ingredients
       

    def get_calories(self):
        calories = []

        for j in range(4):
            calories.append(self.html.findAll('div', class_='emotion-8fp9e2')[j].text) 
            
        return calories
    
    def get_manual(self):
        manual = ''
        
        for point in self.html.findAll('span', class_='emotion-1dvddtv'):
            manual += point.text.replace(u'\xa0', u' ') + '\n' + '\n'
    
        return manual 

    def get_info(self):  
        i = 0
        for recipe in self.html.findAll('div', class_='emotion-1f6ych6'):
            try:
                link = "https://eda.ru"+recipe.find('a', class_='emotion-18hxz5k').get('href')
                photo_dish =  recipe.find('div', class_='emotion-1f9gxzh').find('img', class_='emotion-1bi124x')['src']

                name_dish = recipe.find('a', class_='emotion-18hxz5k').find('span', class_='emotion-1pdj9vu').text
                count_portion = int(re.sub(r'\bпорц[ий|ия|ии]*\b', '', recipe.find('div', class_='emotion-1rwzitj').text).strip())
                cook_time = recipe.find('div', class_='emotion-1cutfi0').text
                category = recipe.find('span', class_='emotion-1h6i17m').text

                calories = int(Parser(Crawler(link).get_soup()).get_calories()[0])
                proteins = int(Parser(Crawler(link).get_soup()).get_calories()[1])
                fats = int(Parser(Crawler(link).get_soup()).get_calories()[2])
                carbohydrates = int(Parser(Crawler(link).get_soup()).get_calories()[3])
                manual = Parser(Crawler(link).get_soup()).get_manual()
                
                ingredients = Parser(Crawler(link).get_soup()).get_ingredients()
        
            
                time.sleep(2)
                
                info = [link, photo_dish, name_dish, count_portion, cook_time, calories, proteins, fats, carbohydrates, manual, category]
        
                print(name_dish) 

                DataBase('eda.db', info, ingredients, link).additional_load_data()
#                 DataBase('eda.db', info, ingredients, link).create_tables_and_insert_data_first()
                i += 1

            except AttributeError:
                print("There's no item with that code")


In [5]:
class DataBase:
    def __init__(self, db_name, info, ingredients, link):
        self.db_name = db_name
        self.info = info 
        self.ingredients = ingredients
        self.link = link
        
    def create_tables(self):
        conn = sqlite3.connect(self.db_name)
        cursor = conn.cursor()

        cursor.execute('''
            CREATE TABLE recipes (
                id INTEGER PRIMARY KEY AUTOINCREMENT,
                link TEXT,
                photo_dish TEXT,
                name_dish TEXT,
                count_portion INTEGER,
                cook_time INTEGER,
                calories INTEGER,
                proteins INTEGER,
                fats INTEGER,
                carbohydrates INTEGER,
                manual TEXT,
                category TEXT)''')

        cursor.execute('''
            CREATE TABLE ingredients (
                id INTEGER PRIMARY KEY AUTOINCREMENT,
                recipe_id INTEGER,
                name_ingredients TEXT,
                count_ingredients TEXT,
                FOREIGN KEY (recipe_id) REFERENCES recipes (id))''')

        conn.commit()

        
    def insert_data(self):
        conn = sqlite3.connect(self.db_name)
        cursor = conn.cursor()
        
        cursor.execute(f"SELECT link FROM recipes WHERE link ='{self.link}'")

        if cursor.fetchone() is None:

            cursor.execute('''
                INSERT INTO recipes
                (link, photo_dish, name_dish, count_portion, cook_time, calories, proteins, fats, carbohydrates, manual, category)
                VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
            ''', (self.info[0],self.info[1],self.info[2],self.info[3],self.info[4],self.info[5],self.info[6],self.info[7],self.info[8],self.info[9],self.info[10]))


            last_recipe_id = cursor.lastrowid

            for ing in self.ingredients:
                cursor.execute('''
                    INSERT INTO ingredients
                    (recipe_id, name_ingredients, count_ingredients)
                    VALUES (?, ?, ?)
                ''', (last_recipe_id, ing, self.ingredients[ing]))

            conn.commit()
          
            
    # Если база данных с таким именем есть - удаление базы данных, если нет - создание новой        
    def create_tables_and_insert_data_first(self):
        conn = sqlite3.connect(self.db_name)
        cursor = conn.cursor()

        table_query = "SELECT name FROM sqlite_master WHERE type='table' AND name='recipes'"
        cursor.execute(table_query)
        if cursor.fetchone() is not None:
            cursor.execute('''DROP TABLE recipes''')
            cursor.execute('''DROP TABLE ingredients''')
            print('такая бд уже есть, удалили таблицы')
            DataBase(self.db_name, self.info, self.ingredients, self.link).create_tables()
            print('таблицы созданы')

        else:  
            DataBase(self.db_name, self.info, self.ingredients, self.link).create_tables()
            print('таблицы созданы')
    
        DataBase(self.db_name, self.info, self.ingredients, self.link).insert_data()

        cursor.close()
    
    # Догрузка данных
    def additional_load_data(self):
        conn = sqlite3.connect(self.db_name)
        cursor = conn.cursor()
        
        DataBase(self.db_name, self.info, self.ingredients, self.link).insert_data()
        
        cursor.close()
 


In [6]:
class C:
    def main(page):
        i = page
        while True:
            print(i)
#             html = Crawler(f'https://eda.ru/recepty/sousy-marinady?page={i}').get_soup()
#             Parser(html).get_info()
#             html = Crawler(f'https://eda.ru/recepty/napitki?page={i}').get_soup()
#             Parser(html).get_info()
#             html = Crawler(f'https://eda.ru/recepty/zavtraki?page={i}').get_soup()
#             Parser(html).get_info()
#             html = Crawler(f'https://eda.ru/recepty/salaty?page={i}').get_soup()
#             Parser(html).get_info()
#             html = Crawler(f'https://eda.ru/recepty/supy?page={i}').get_soup()
#             Parser(html).get_info() 
#             html = Crawler(f'https://eda.ru/recepty/pasta-picca?page={i}').get_soup()
#             Parser(html).get_info()
#             html = Crawler(f'https://eda.ru/recepty/zakuski?page={i}').get_soup()
#             Parser(html).get_info()
            
            html = Crawler(f'https://eda.ru/recepty/vypechka-deserty?page={i}').get_soup()
            Parser(html).get_info()
            
            i += 1

In [ ]:
https://eda.ru/recepty/osnovnye-blyuda?page=522
https://eda.ru/recepty/vypechka-deserty?page=149
# супы 45
# закуски 54
# паста 21
# салаты 42
# ризотто 3
# напитки 16
# заготовки 2
# сэндвичи 6
# соусы 8
# бульоны 2
# завтраки 34

13 категорий всего

76 страница

In [ ]:
if __name__ == '__main__':
    C.main(211)

211
Творожные кексы с клюквой
Специи для тыквенного пирога
Ванильный кекс с влажной шоколадной начинкой
Рождественские маффины
Арбузный шербет с водкой
Кофейный рулет
Мороженое из варенья
Быстрая творожная запеканка с манкой
Лимонный крем
Капустные оладьи с зеленью
Датское твердое печенье
Французская меренга
Овсяные банановые кексы с семечками
Слоеный пирог с яблоками и миндалем по-нормандски
212
Постное шоколадное печенье без выпечки
Французская меренга
Пареные яблоки «Дофин»
Пирог «Бешамель»
Шоколадно-малиновые замороженные пирожные
Оладьи из мацы
Курица в блинах
Жареные пирожки с горохом
Бисквитный пирог с нектаринами
Яблочные маффины
Торт с грецкими орехами
Хачапури с тыквой
Ванильный пудинг с джемом из свежей клубники
Открытый пирог с черной смородиной
213
Шоколадные трюфели с лимоном и тимьяном
Мороженое «Бальзам»
Печенье «Черепица»
Яблоки в духовке
Блины гречневые тонкие
Лимонно-медовый пасхальный пирог
Блинный торт с бананами и банановым кремом
Печенье польворонес
Песочный шоко

In [95]:
with sqlite3.connect('eda.db') as connection:
    cursor = connection.cursor()
    cursor.execute("SELECT * FROM ingredients ")
    rows = cursor.fetchall()
    for row in rows:
        print(row)    

In [94]:
with sqlite3.connect('eda.db') as connection:
    cursor = connection.cursor()
    cursor.execute("DELETE FROM ingredients")
    rows = cursor.fetchall()
    for row in rows:
        print(row) 

In [22]:
conn = sqlite3.connect('eda.db')
cursor = conn.cursor()

# создаем два пустых массива
manuals = []
categories = []

# получаем все значения manual и category из базы данных
for row in cursor.execute('SELECT manual, category FROM recipes'):
    manuals.append(row[0])
    categories.append(row[1])

# закрываем соединение с базой данных
conn.close()

In [24]:
categories

['Выпечка и десерты',
 'Завтраки',
 'Паста и пицца',
 'Выпечка и десерты',
 'Паста и пицца',
 'Выпечка и десерты',
 'Выпечка и десерты',
 'Основные блюда',
 'Салаты',
 'Выпечка и десерты',
 'Выпечка и десерты',
 'Выпечка и десерты',
 'Завтраки',
 'Супы',
 'Выпечка и десерты',
 'Основные блюда',
 'Завтраки',
 'Закуски',
 'Основные блюда',
 'Завтраки',
 'Основные блюда',
 'Супы',
 'Выпечка и десерты',
 'Паста и пицца',
 'Выпечка и десерты',
 'Супы',
 'Выпечка и десерты',
 'Завтраки',
 'Супы',
 'Завтраки',
 'Выпечка и десерты',
 'Супы',
 'Супы',
 'Закуски',
 'Выпечка и десерты',
 'Выпечка и десерты',
 'Завтраки',
 'Салаты',
 'Основные блюда',
 'Супы',
 'Закуски',
 'Основные блюда',
 'Супы',
 'Салаты',
 'Супы',
 'Паста и пицца',
 'Салаты',
 'Выпечка и десерты',
 'Выпечка и десерты',
 'Выпечка и десерты',
 'Выпечка и десерты',
 'Основные блюда',
 'Паста и пицца',
 'Салаты',
 'Салаты',
 'Выпечка и десерты',
 'Супы',
 'Супы',
 'Выпечка и десерты',
 'Выпечка и десерты',
 'Выпечка и десерты',
 

In [28]:
# создаем словарь, где ключами будут названия категорий, а значениями - их номера
category_dict = {}
categories_id = []

# проходимся по каждой категории в списке
for category in categories:
    # если категория уже есть в словаре, то добавляем ее номер в список categories_id
    if category in category_dict:
        categories_id.append(category_dict[category])
    # если категории еще нет в словаре, то добавляем ее и присваиваем ей новый номер
    else:
        category_dict[category] = len(category_dict)
        categories_id.append(category_dict[category])

print(categories_id)

[0, 1, 2, 0, 2, 0, 0, 3, 4, 0, 0, 0, 1, 5, 0, 3, 1, 6, 3, 1, 3, 5, 0, 2, 0, 5, 0, 1, 5, 1, 0, 5, 5, 6, 0, 0, 1, 4, 3, 5, 6, 3, 5, 4, 5, 2, 4, 0, 0, 0, 0, 3, 2, 4, 4, 0, 5, 5, 0, 0, 0, 1, 3, 3, 3, 0, 7, 3, 8, 5, 3, 0, 7, 4, 5, 0, 3, 0, 1, 0, 4, 0, 3, 3, 0, 0, 3, 9, 0, 3, 1, 3, 0, 4, 3, 6, 8, 3, 0, 5, 0, 3, 3, 3, 0, 5, 0, 3, 0, 6, 3, 0, 6, 1, 0, 1, 2, 3, 5, 5, 6, 1, 0, 0, 0, 4, 0, 4, 6, 5, 3, 0, 3, 2, 6, 3, 3, 6, 0, 3, 4, 2, 0, 6, 3, 4, 5, 5, 6, 0, 2, 3, 0, 3, 0, 3, 0, 5, 3, 3, 3, 1, 0, 1, 1, 4, 2, 1, 6, 0, 0, 5, 2, 0, 6, 3, 6, 5, 0, 5, 1, 0, 0, 1, 0, 3, 0, 3, 0, 1, 0, 6, 0, 2, 1, 3, 3, 1, 4, 2, 0, 5, 3, 3, 1, 6, 4, 5, 0, 2, 7, 0, 1, 1, 10, 0, 5, 3, 0, 3, 0, 5, 3, 5, 0, 3, 2, 0, 3, 3, 0, 3, 0, 0, 0, 2, 0, 4, 3, 3, 5, 6, 6, 0, 0, 5, 6, 0, 1, 1, 3, 5, 6, 0, 2, 3, 5, 4, 6, 1, 3, 5, 0, 3, 1, 2, 0, 0, 5, 3, 1, 0, 9, 3, 3, 3, 6, 6, 3, 4, 3, 1, 1, 1, 8, 5, 1, 0, 1, 4, 3, 1, 3, 0, 3, 1, 3, 0, 4, 3, 10, 0, 3, 0, 6, 4, 0, 3, 9, 0, 5, 3, 3, 1, 5, 0, 6, 6, 5, 3, 4, 1, 5, 3, 6, 5, 3, 6, 5, 3, 2, 3, 5